# Modified IQAE tests

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
import yaml
import csv
import datetime
import itertools
from tqdm import tqdm

from random import sample, seed
from collections import defaultdict

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, assemble
from qiskit.algorithms import amplitude_estimators, EstimationProblem
from qiskit.algorithms import IterativeAmplitudeEstimation as BaseIterativeAmplitudeEstimation

from algorithms import IterativeAmplitudeEstimation, ModifiedIterativeAmplitudeEstimation
from algorithms import NoQuantumIterativeAmplitudeEstimation
from operators import *

In [2]:
# load config 

with open('./config.yaml') as f:
    config = yaml.safe_load(f)
    
# add default arguments if not already there
defaults = {
    'experiment_name': '{}_modified-iqae',
    'results_path': './results/{}',
    'simulator': 'aer_simulator',
    'compare': True,
    'noise': 0.0,
    'plots': True,
    'verbose': False,
}

now = datetime.datetime.now(datetime.timezone.utc).strftime('%Y-%m-%dT%H%M%SZ')
defaults['experiment_name'] = defaults['experiment_name'].format(now)
defaults['results_path'] = defaults['results_path'].format(defaults['experiment_name'])

for key in defaults:
    if key not in config:
        config[key] = defaults[key]

# process parameters from config

runs = config['runs']
shots = config['shots']
N = config['a_resolution']
step = config['a_step']
alpha = config['alpha']
epsilons = config['epsilons']
confint_method = config['confint_method']
noise = config['noise']
experiment_name = config['experiment_name']
results_path =  config['results_path']
simulator = config['simulator']
compare = config['compare']
plots = config['plots']
verbose = config['verbose']

# maybe some other error checking for when it's done
# especially for ensuring amplitude is [0,1]

if isinstance(step, int):
    amplitudes = np.arange((N // step)+1) * step
else:
    amplitudes = step
    
epsilons = [float(eps) for eps in epsilons]

if confint_method == 'all':
    methods = ['chernoff', 'beta']
else:
    methods = [confint_method]

algs = ['miae']
if compare:
    algs.append('iae')
    
results_per_round_path = os.path.join(results_path, 'per_round')

if not os.path.exists('./results'):
    os.mkdir('./results')
if not os.path.exists(results_path):
    os.mkdir(results_path)
if not os.path.exists(results_per_round_path):
    os.mkdir(results_per_round_path)

## Compare Modified IQAE to No-Quantum IQAE

In [3]:
# define the estimation problem and oracle function
def make_problem(n, marked):
    
    def good_state(state):
        bin_marked = [(n-len(bin(s))+2)*'0'+bin(s)[2:] for s in marked]
        return (state in bin_marked)

    problem = EstimationProblem(
        state_preparation=A(n),        # A operator
        grover_operator=Q(n, marked),  # Q operator
        objective_qubits=range(n),
        is_good_state=good_state       # the "good" state Psi1 is identified as measuring |1> in qubit 0
    )
    
    return problem

# process the per-round info
def process_state(state, verbose=False):
    if verbose:
        for k,v in state.items():
            print(k)
            print(v)
    if len(state) == 0: return [],[],[]
    round_shots = state['round_shots']
    queries = state['n_queries']
    removed = False

    if 0 in round_shots:
        shots_at_k0 = round_shots.pop(0)
        removed = True
    if 0 in queries:
        queries_at_k0 = queries.pop(0)

    k_i = [k for k in round_shots]
    queries_i = [queries[k] for k in k_i]
    shots_i = ([shots_at_k0] if removed else []) + [round_shots[k] for k in k_i]

    if removed:
        k_i.insert(0, 0.1)

    return shots_i, queries_i, k_i

In [4]:
def experiment(alg: str, shots: int, M: int, N: int, alpha: float, 
               epsilon: float, confint_method: str, noise: float, 
               runs: int, simulator: str):

    AE = None
    if alg == 'miae':
        AE = ModifiedIterativeAmplitudeEstimation(epsilon_target=epsilon, 
                                                  alpha=alpha, 
                                                  confint_method=confint_method, 
                                                  quantum_instance=simulator)

    else: # alg == 'iae'
        AE = IterativeAmplitudeEstimation(epsilon_target=epsilon, 
                                          alpha=alpha,
                                          confint_method=confint_method, 
                                          quantum_instance=simulator)

        
    n = int(np.log2(N))
    marked = sample(range(N), M)
    problem = make_problem(n, marked)
    ae_results = []

    for _ in range(runs):        
    
        # setup
        state = defaultdict(dict)
        scaling = 1 # figure out what to do here

        if noise > 0:
            N = max(128, N * scaling) # replace with the correct formula
            n = int(np.log2(N))
            sampled_noise = np.random.uniform(-noise if M > N else 0, noise if M < N else 0)
            scaled_noisy_M = int(max(0, min(N, (M + sampled_noise) * scaling)))

            marked = sample(range(N), scaled_noisy_M)
            problem = make_problem(n, marked)
            
            # scale back to original resolution
            noisy_M = scaled_noisy_M / scaling
            N = config['a_resolution']
            n = int(np.log2(N))
            
        # run estimation
        ae_result = AE.estimate(problem,
                                shots=shots,
                                ground_truth=noisy_M/N if noise > 0 else M/N,
                                state=state,
                                verbose=verbose)
        
        ae_results.append(ae_result)

        if verbose: print()

        # round-specific logs
        round_shots, round_queries, round_k = process_state(state)

        round_results_csv = os.path.join(results_per_round_path, f'{M/N}_{confint_method}_{epsilon:.1e}_{alg}_round-results.csv')

        with open(round_results_csv, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['round_shots'] + round_shots)
            writer.writerow(['round_queries'] + round_queries)
            writer.writerow(['round_k'] + round_k)

    # experiment-specific logs
    half_ci_widths = np.array([(res.confidence_interval_processed[1] - res.confidence_interval_processed[0]) / 2 for res in ae_results])
    queries = np.array([res.num_oracle_queries for res in ae_results])
    estimations = np.array([res.estimation for res in ae_results])

    results_csv = os.path.join(results_path, f'{M/N}_{confint_method}_{alg}_results.csv')

    with open(results_csv, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epsilon] + queries.tolist())
    
    if verbose:
        print(f'Results using algorithm \'{alg}\', amplitude {M}/{N} = {M/N}', end='')
        print(f' (with noise: {noisy_M/N})' if noise > 0 else '', end=' ')
        print(f'averaged over {runs} runs:')
        print(f'\tEstimation: {round(estimations.mean(), 10)}')
        print(f'\tTotal queries: {round(queries.mean())}')
        print(f'\tCI width: {half_ci_widths.mean()}')

        # for i, epsilon in enumerate(epsilons):
            
        #     if plots:
        #         fig,axs = plt.subplots(1,3,figsize=(15,5))

        #         fig.suptitle(f'a={ki/N}, eps={epsilon}')
        #         axs[0].set_xscale('log')
        #         axs[0].set_title('Shots_i vs. k_i')

        #         axs[1].set_xscale('log')
        #         axs[1].set_yscale('log')
        #         axs[1].set_title('Queries_i vs. k_i')

        #         axs[2].set_yscale('log')
        #         axs[2].set_title('k_i vs. i')

        #     n_entries = 0
        #     for confint_method in ['chernoff', 'beta']:

        #         if verbose:
        #             print('ε:',epsilon)

                # configure the number of shots this way and pray that it works
        

        # for recording intermediate algo results
        

#                 miae_result = MIAE.estimate(problem,
#                                             state=state,
#                                             verbose=verbose)

#                 iae_result = IAE.estimate(problem2,
#                                           state=state2,
#                                           verbose=verbose)

#                 miae_results[confint_method].append(miae_result)
#                 iae_results[confint_method].append(iae_result)

#                 if verbose:
#                     print()

                
#                 base_shots, base_queries, base_k = process_state(state2)

#                 if plots:
#                      # plots for shots vs k
#                     axs[0].plot(mod_k, mod_shots, label=f'{confint_method.title()} modified')
#                     axs[0].scatter(mod_k, mod_shots)
#                     axs[0].plot(base_k, base_shots, label=f'{confint_method.title()} base')
#                     axs[0].scatter(base_k, base_shots)

#                     # plots for nqueries vs k
#                     axs[1].plot(mod_k, mod_queries, label=f'{confint_method.title()} modified')
#                     axs[1].scatter(mod_k, mod_queries)
#                     axs[1].plot(base_k[1:], base_queries, label=f'{confint_method.title()} base')
#                     axs[1].scatter(base_k[1:], base_queries)

#                     # plots for k
#                     axs[2].plot(mod_k, label=f'{confint_method.title()} modified')
#                     axs[2].scatter(range(len(mod_k)), mod_k)
#                     axs[2].plot(base_k, label=f'{confint_method.title()} base')
#                     axs[2].scatter(range(len(base_k)), base_k)


#                 n_entries = max(len(base_k), len(mod_k), n_entries)

#             if plots:
#                 for i in range(3): axs[i].legend()
#                 axs[2].plot(range(n_entries), np.repeat(np.pi / 4 / epsilon, n_entries), c='r', linestyle='--')

        
#         if plots:
#             # process results
#             plt.figure(figsize=(15,7))
#             plt.yscale('log')
#         #     plt.ylim(10**4, 10**10)
#             plt.xscale('log')

#             plt.xlim(epsilons[0]*2, epsilons[-1]/2)
#             plt.title(f'{"Noisy " if add_noise else ""}Error vs. Number of Queries')

#         miae_epsilon_i = {}
#         miae_nshots_i = {}
#         iae_epsilon_i = {}
#         iae_nshots_i = {}
        
#         for confint_method in ['chernoff', 'beta']:
#             miae_epsilon_i[confint_method] = [(res.confidence_interval_processed[1] - res.confidence_interval_processed[0]) / 2 for res in miae_results[confint_method]]
#             miae_nshots_i[confint_method]  = [res.num_oracle_queries for res in miae_results[confint_method]]

#             iae_epsilon_i[confint_method] = [(res.confidence_interval_processed[1] - res.confidence_interval_processed[0]) / 2 for res in iae_results[confint_method]]
#             iae_nshots_i[confint_method]  = [res.num_oracle_queries for res in iae_results[confint_method]]

#             if verbose:
#                 print(f'a: {ki}/{2**n} = {ki/2**n}')
#                 print(f'{exprs[0]} estimations:', [res.estimation for res in miae_results[confint_method]])
#                 print(f'{exprs[1]} estimations:', [res.estimation for res in iae_results[confint_method]])

#             miae_total_queries, iae_total_queries = sum(miae_nshots_i[confint_method]), sum(iae_nshots_i[confint_method])
            
#             if verbose:
#                 print(f'{exprs[0]} total queries:', miae_total_queries)
#                 print(f'{exprs[1]} total queries:', iae_total_queries)

#                 print(f'{exprs[0]} epsilons (CI width):', miae_epsilon_i)
#                 print(f'{exprs[1]} epsilons (CI width):', iae_epsilon_i)

#             diff = round(abs(iae_total_queries - miae_total_queries) / miae_total_queries * 100, 2)
#         #     print('Modified version wins?', iae_total_queries < miae_total_queries, f'with {diff}% difference')

#             wins += int(iae_total_queries < miae_total_queries)
#             matches += 1

#             if plots:
#                 # plots for query complexity, shots per k

#                 # plot query complexity
#                 plt.scatter(epsilons, miae_nshots_i[confint_method])
#                 plt.plot(epsilons, miae_nshots_i[confint_method], label=f'{confint_method.title()} modified')
#                 plt.scatter(epsilons, iae_nshots_i[confint_method])
#                 plt.plot(epsilons, iae_nshots_i[confint_method], label=f'{confint_method.title()} base')

#         if plots:
#             plt.legend()
#             plt.show()

#         # save results for epsilon vs nshots
#         miae_nshots.append(miae_nshots_i)
#         miae_epsilon.append(miae_epsilon_i)
#         iae_nshots.append(iae_nshots_i)
#         iae_epsilon.append(iae_epsilon_i)

#         ki += 1

# #     iae_nshots = np.array(iae_nshots)
    
#     return { 
#         'miae_nshots': miae_nshots, 
#         'miae_epsilon': miae_epsilon,
#         'iae_nshots': iae_nshots,
#         'iae_epsilon': iae_epsilon
#     }

    # print('% modified > 3x modified:', wins/matches)

In [5]:
experiment_combos = list(itertools.product(algs, amplitudes, epsilons, methods))
print('Experiment configuration size:', len(experiment_combos))
print('# runs per configuration:', runs)
print('Total runs:', len(experiment_combos) * runs)

# re-save the config into the file
with open(os.path.join(results_path, 'config.yaml'), 'w') as out:
    yaml.dump(config, out)
    
for alg, M, epsilon, method in tqdm(experiment_combos):
    experiment(alg, shots, M, N, alpha, epsilon, method, noise, runs, simulator)
    

  0%|          | 0/544 [00:00<?, ?it/s]

Experiment configuration size: 544
# runs per configuration: 100
Total runs: 54400


100%|██████████| 544/544 [08:39<00:00,  1.05it/s]


# Query complexity

In [6]:
# amplitudes = np.arange(k+1)

# for a in amplitudes:
#     lines = []
    
#     plt.figure(figsize=(15,7))
#     plt.yscale('log')
# #     plt.ylim(10**4, 10**10)
#     plt.xscale('log')

#     plt.xlim(epsilons[0]*2, epsilons[-1]/2)
#     plt.title(f'Error vs. Number of Queries, a={a}/{k}')
    
#     for expr in ['miae', 'iae']:
#         for confint in ['chernoff', 'beta']:
#             queries = np.array([[res_expr_i[confint] for res_expr_i in res_i[f'{expr}_nshots']][a] for res_i in res]).mean(axis=0)
#             plt.scatter(epsilons, queries)
#             line, = plt.plot(epsilons, queries, label=f'{confint.title()} {expr}')
#             lines.append(line)
    
#     plt.legend()
    
#     plt.savefig(f'results_images/err-vs-queries_{a}-{k}.png')
#     # plt.show()

# Query count vs. input amplitude

In [7]:
# for expr in ['miae_nshots', 'iae_nshots']:
#     for confint in ['chernoff', 'beta']:
        
#         plt.figure()
#         arr = np.array([[res_expr_i[confint] for res_expr_i in res_i[expr]] for res_i in res]).mean(axis=0)

#         lines = []
#         for i,eps in enumerate(epsilons):
#             line, = plt.plot(2**np.arange(arr.shape[0]), arr[:,i], label='{:.0e}'.format(eps))
#             plt.scatter(2**np.arange(arr.shape[0]), arr[:,i])
#             lines.append(line)

#         plt.title(f'Number of queries vs. 1/Input amplitude, {expr} + {confint}')
#         plt.legend(handles=lines)
#         plt.xscale('log')
#         plt.yscale('log')

#         # plt.show()
#         plt.savefig(f'results_images/queries-vs-a_{expr}-{confint}')